In [1]:
import praw
import pandas as pd
import datetime as dt

In [2]:
# Logging into api
reddit = praw.Reddit(client_id='Arq_pDXruFQ4Ag', \
                     client_secret='L00q3WDmmlOYnXuHyEBYl8od5jg', \
                     user_agent='ebicher-test', \
                     username='sencinitas', \
                     password='galaxy123')

In [3]:
# Grabbing Chargers subreddit
subreddit = reddit.subreddit('Chargers')

In [4]:
# Looking at methods
subreddit?

In [6]:
# Top 20 Submissions
top_subreddit = subreddit.top(limit=1000)

In [7]:
# Top submission title
for submission in subreddit.top(limit=1):
    print(submission.title, submission.id)

TRAITOR. Let's make this the #1 image on Google! 5nlkzv


In [8]:
# Creating dictionary
topics_dict = { "title":[],
                "score":[],
                "id":[], 
                "url":[], 
                "comms_num": [],
                "created": [],
                "body":[],
                "subreddit": []}


In [9]:
# Filling dictionary with data
for submission in top_subreddit:
    topics_dict["title"].append(submission.title)
    topics_dict["score"].append(submission.score)
    topics_dict["id"].append(submission.id)
    topics_dict["url"].append(submission.url)
    topics_dict["comms_num"].append(submission.num_comments)
    topics_dict["created"].append(submission.created)
    topics_dict["body"].append(submission.selftext)
    topics_dict["subreddit"].append(submission.subreddit)

In [10]:
# Putting it into a dataframe
topics_data = pd.DataFrame(topics_dict)

In [11]:
# Data
topics_data.head()

,title,score,id,url,comms_num,created,body,subreddit
0,TRAITOR. Let's make this the #1 image on Google!,8852,5nlkzv,https://i.redd.it/m0wnou0j6c9y.png,198,1.484280e+09,,Chargers
1,Raiders. Up vote this so that when people give...,3036,62tetg,https://i.redd.it/65j94evunyoy.jpg,33,1.491090e+09,,Chargers
2,Asshole,2170,5ngy1c,http://imgur.com/a/MInCI,136,1.484220e+09,,Chargers
3,"Brochachos, Pay your internet provider a separ...",1457,7ene7f,https://www.battleforthenet.com/,38,1.511346e+09,,Chargers
4,Charger fire McCoy,1349,5likbj,https://twitter.com/greggrosenthal/status/8157...,171,1.483350e+09,,Chargers


In [12]:
topics_data.drop_duplicates(subset=['id'], inplace=True)

In [13]:
# Method for getting date
def get_date(created):
    return dt.datetime.fromtimestamp(created)

In [14]:
_timestamp = topics_data["created"].apply(get_date)

In [15]:
topics_data = topics_data.assign(timestamp = _timestamp)

In [16]:
topics_data.head()

,title,score,id,url,comms_num,created,body,subreddit,timestamp
0,TRAITOR. Let's make this the #1 image on Google!,8852,5nlkzv,https://i.redd.it/m0wnou0j6c9y.png,198,1.484280e+09,,Chargers,2017-01-12 19:55:34
1,Raiders. Up vote this so that when people give...,3036,62tetg,https://i.redd.it/65j94evunyoy.jpg,33,1.491090e+09,,Chargers,2017-04-01 16:34:33
2,Asshole,2170,5ngy1c,http://imgur.com/a/MInCI,136,1.484220e+09,,Chargers,2017-01-12 03:19:56
3,"Brochachos, Pay your internet provider a separ...",1457,7ene7f,https://www.battleforthenet.com/,38,1.511346e+09,,Chargers,2017-11-22 02:22:44
4,Charger fire McCoy,1349,5likbj,https://twitter.com/greggrosenthal/status/8157...,171,1.483350e+09,,Chargers,2017-01-02 01:39:52


In [85]:
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords

In [86]:
submission = reddit.submission(id='a0btbm')

In [151]:
from praw.models import MoreComments
submission.comments.replace_more(limit=None)
list_of_comments = []
comment_queue = submission.comments[:]  # Seed with top-level
while comment_queue:
    comment = comment_queue.pop(0)
    list_of_comments.append(comment.body)
    comment_queue.extend(comment.replies)

In [166]:
df = pd.DataFrame(list_of_comments, label='body')
df

TypeError: __init__() got an unexpected keyword argument 'label'

In [153]:
from gensim.parsing.preprocessing import remove_stopwords

# Utility function to clean text.
def text_cleaner(text):
    listy = []
    
    for i in text:
    # Visual inspection shows spaCy does not recognize the double dash '--'.
    # Better get rid of it now!
        j = re.sub(r'--',' ',i)
    # Get rid of headings in square brackets.
        j = re.sub("[\[].*?[\]]", "", i)
    # Get rid of extra whitespace.
        j = ' '.join(i.split())
    # Remove stopwords
        j = remove_stopwords(j)
        j = j.lower()
        listy.append(j)
    return listy

clean_comments = text_cleaner(list_of_comments)

In [154]:
clean_comments[1].split()

['if', 'don’t', 'win', 'game', 'ima', 'eat', 'lettuce', 'end']

In [156]:
len(clean_comments)

2199

In [157]:
from gensim.models import word2vec
model = word2vec.Word2Vec([c.split() for c in clean_comments], size=1000, window=3, min_count=2, workers=4)

In [158]:
vocab = model.wv.vocab.keys()

In [159]:
print(model.wv.most_similar(positive=['chargers']))

[('i', 0.5796991586685181), ('like', 0.5622677206993103), ('play', 0.5541449189186096), ('team', 0.5519105792045593), ('good', 0.5378639698028564), ('game', 0.5258690714836121), ('better', 0.5013057589530945), ('it’s', 0.49876493215560913), ('start', 0.49432626366615295), ('let', 0.49037477374076843)]


/home/ebicher/.local/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [160]:
model.most_similar("catch")

/home/ebicher/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/home/ebicher/.local/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('play', 0.3908742666244507),
 ('i', 0.37914562225341797),
 ('start', 0.3776373565196991),
 ('team', 0.3751877546310425),
 ('good', 0.36530205607414246),
 ('it’s', 0.3384231626987457),
 ('like', 0.33778661489486694),
 ('season', 0.33644410967826843),
 ('rivers', 0.33623939752578735),
 ('gordon', 0.3351577818393707)]

In [59]:
from gensim.parsing.preprocessing import remove_stopwords

def rem_stop(listtext):
    listy = []
    for i in listtext:
    # Get rid of headings in square brackets.
        text = re.sub("[\[].*?[\]]", "", text)
    # Get rid of extra whitespace.
        text = ' '.join(text.split())
        listy.append(remove_stopwords(i))
    return listy

In [ ]:
import gensim
model = gensim.models.Word2Vec(
    listoftitles,
    size=150,
    window=10,
    min_count=2,
    workers=10)
model.train(list_of_comments, total_examples=len(list_of_comments), epochs=10)

In [83]:
vocab = model.wv.vocab.keys()

In [84]:
vocab

dict_keys(['game', 'If', 'we', 'this', 'just', 'some', 'and', 'end', 'it', 'all', 'WITH', 'A', 'IS', 'THE', '19/19', 'for', 'Rivers', 'is', 'right', 'now', 'playing', 'out', 'of', 'his', 'in', 'Melvin', 'a', 'are', 'every', 'week', 'yards', 'on', 'that', 'how', 'have', 'at', 'the', 'other', 'no', 'lol.', 'Phil', 'as', 'he', 'first', 'quarter', 'MY', 'FUCK', 'I', 'to', 'be', 'It', 'was', 'not', 'do', 'thread', 'Gordon', 'still', 'being', 'an', 'like', 'fucking', 'if', 'up', 'know', 'us', 'going', 'get', 'BOSA', 'walking', 'off', 'field', 'they', "I'm", 'injury', 'but', 'team', 'very', 'him', 'back', 'guys', 'run', 'our', 'love', 'there', 'Philip', 'or', 'running', 'Chargers', 'QB', 'Bosa', 'would', 'really', 'play', 'Geno', 'so', 'you', 'gets', 'these', 'me', 'Man', 'hope', 'We', 'need', 'next', 'week.', 'when', 'think', 'could', 'after', 'see', 'Good', 'feel', 'against', 'now.', 'about', 'Not', 'let', 'around', 'last', 'today.', 'season', 'over', 'with', 'fuck', 'Steelers', 'looks', 'g